



# CSE213 - Numerical Analysis

# Lab 11 - Multi-step Methods for Solving ODEs: Adams-Bashforth Corrector-Predictor Method

The Adams-Bashforth method is an explicit method for solving ODEs. It is a multi-step method that uses the values of the function and its derivatives at previous time steps to predict the value at the next time step. The basic idea is to use the Taylor series expansion of the function and its derivatives to predict the value at the next time step. The method can be used to obtain high-order accuracy, but it is also subject to stability issues. The Adams-Bashforth predictor is as follows:
$$\large
y_{n+1, p}=y_n+\frac{h}{24}\left(55 y_n^{\prime}-59 y_{n-1}^{\prime}+37 y_{n-2}^{\prime}-9 y_{n-3}^{\prime}\right)\text{.}
$$
For $n=3$, we have
$$\large
y_{4,p}=y_{3}+\frac{h}{24}\Bigl(55y_{3}^{\prime}-59y_{2}^{\prime}+37y_{1}^{\prime}-9y_{0}^{\prime}\Bigr)\text{.}$$

The Adams-Bashforth method can be combined with a corrector step to improve its accuracy. The corrector step uses the predicted value to estimate the error, and then adjusts the prediction to obtain a more accurate estimate. This is known as the Adams-Bashforth corrector predictor method. The Adams-Bashforth corrector formula is
$$\large y_{n+1,c}=y_{n}+\frac{h}{24}\left(9y_{n+1}^{\prime}+19y_{n}^{\prime}-5y_{n-1}^{\prime}+y_{n-2}^{\prime}\right)\text{.}$$

Putting $n =3$, we get

$$\large y_{4,c}=y_{3}+\frac{h}{24}\Bigl(9y_{4}^{\prime}+19y_{3}^{\prime}-5y_{2}^{\prime}+y_{1}^{\prime}\Bigr)\text{.}$$

In this lab, you will implement the Adams-Bashforth corrector-predictor algorithm to approximate the solution of a given ordinary differential equation. You will start by writing the predictor algorithm, which uses a finite difference approximation to predict the next value of the solution. Then, you will implement the corrector algorithm, which refines the prediction by using a weighted average of the previous approximations. Finally, you will test your implementation on a simple example problem and compare the results to the exact solution.

### Exercise 1

Implement the predictor algorithm for the Adams-Bashforth method up to order 4. You should be able to use this predictor algorithm for any given initial condition and time step size. Write a function that takes as input
- the current time
- the current approximation of the solution
- the time step size,

and returns the predicted value of the solution at the next time step.

In [3]:
import numpy as np
import matplotlib.pyplot as plt

def predictor(f, t, y, h, f_history):
    """
    Implement the predictor formula of the Adams-Bashforth algorithm of the 4th order
    Inputs:
      - f: the function f(t, y) defining the differential equation
      - t: the current time
      - y: the current value of the dependent variable y
      - h: the step size
      - f_history: array of the last three f(t, y) evaluations
    Output:
      - y_pred: the predicted value of y at the next time step
    """
    y_pred = y + ((h / 24) * ((55 * f_history[-1]) - (59 * f_history[-2]) + (37 * f_history[-3]) - (9 * f_history[-4]))) 
    return y_pred

Implement the corrector algorithm for the Adams-Bashforth method of order 4, which can be used for any given initial condition, time step size, and predicted value of the solution. To achieve this, write a function that takes the current time, current approximation of the solution, predicted value of the solution at the next time step, and time step size as inputs and returns the corrected value of the solution at the next time step. This function can be used to improve the accuracy of the solution obtained through the Adams-Bashforth method.

In [7]:
def corrector(f, t, y, h, y_pred, f_history):
    """
    Implement the corrector formula of the Adams-Moulton algorithm of the 4th order
    Inputs:
      - f: the function f(t, y) defining the differential equation
      - t: the current time
      - y: the current value of the dependent variable y
      - h: the step size
      - y_pred: the predicted value of y at the next time step
      - f_history: array of the last two f(t, y) evaluations
    Output:
      - y_corr: the corrected value of y at the next time step
    """
    y_corr = y + ((h / 24) * ((9 * f(t, y_pred)) + (19 * f_history[-1]) - (5 * f_history[-2]) + (1 * f_history[-3]))) 
    return y_corr

### Exercise 2

Implement the Adams-Bashforth Corrector-Predictor algorithm, and use any single-step method for the first few steps.

In [63]:
def RK4(f, t, y, h):
    """
    Implement the Runge-Kutta fourth order algorithm over the starting values
    Inputs:
      - f: the function f(t, y) defining the differential equation
      - t: the current time
      - y: the current value of the dependant variable y
      - h: the step size
    Output:
      - y_approximate: the approximated y at the starting values 
    """
    k1 = f(t, y)
    k2 = f(t + h/2, y + (h/2)*k1)
    k3 = f(t + h/2, y + (h/2)*k2)
    k4 = f(t + h, y + (h*k3))

    y_approximate = y + (h/6 * (k1 + 2*k2 + 2*k3 + k4))
    return y_approximate

def adams_bashforth_corrector(f, t0, y0, h, t_end):
    """
    Implement the Adams-Bashforth Predictor-Corrector algorithm of the 4th order
    Inputs:
      - f: the function f(t, y) defining the differential equation
      - t0: the initial time
      - y0: the initial value of the dependent variable y
      - h: the step size
      - t_end: the end time
    Output:
      - t: an array of the time steps
      - y: an array of the values of the dependent variable y
    """
    # for starters
    steps = int((t_end - t0) / h)
    t = [(0.1 * i) for i in range(steps + 1)]
    y = np.zeros(steps + 1)
    f_history = list(np.zeros(4,))
    y[0] = y0
    f_history[0] = f(t[0], y[0])
    
    for i in range(1, 4):
       y[i] = RK4(f, t[i - 1], y[i - 1], h)
       f_history[i] = f(t[i], y[i])
    
    # For Adam-Bashforth loop
    for n in range(3, steps):
       y_pred = predictor(f, t[n], y[n], h, f_history)
       y_corr = corrector(f, t[n + 1], y[n], h, y_pred, f_history)
       y[n + 1] = y_corr
       f_history.append(f(t[n + 1], y[n + 1]))
       f_history.pop(0)
    return t, y

### Testing

In [66]:
import numpy as np

def f(t, y): return -y

# Test case
t0, y0, t_end, h = 0, 1, 2, 0.1

t, y = adams_bashforth_corrector(f, t0, y0, h, t_end)

y_expected = [1.0, 0.9048374, 0.8187308, 0.7408182, 0.67032, 0.6065307, 0.5488116, 0.4965853,
              0.449329, 0.4065697, 0.3678794, 0.3328711, 0.3011942, 0.2725318, 0.246597,
              0.2231302, 0.2018965, 0.1826835, 0.1652989, 0.1495686, 0.1353353]

t_expected = [0.1*i for i in range(21)]

# Assert statements
assert np.isclose(t[0], 0)
assert np.isclose(y[0], 1)
assert np.isclose(t[-1], 2)
assert np.isclose(y[-1], y_expected[-1])
assert len(t) == len(y)
assert np.allclose(y, y_expected, atol=1e-6)
assert np.allclose(t, t_expected, atol=1e-6)